In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/who-is-the-real-winner/sample_submission.csv
/kaggle/input/who-is-the-real-winner/train.csv
/kaggle/input/who-is-the-real-winner/test.csv


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder


In [51]:
# Load training and test datasets
train_data = pd.read_csv("/kaggle/input/who-is-the-real-winner/train.csv")
test_data = pd.read_csv("/kaggle/input/who-is-the-real-winner/test.csv")

In [52]:
# Define conversion factors
crore_factor = 10000000  # 1 crore = 10,000,000 rupees
lac_factor = 100000  # 1 lac = 100,000 rupees
thou_factor = 1000  # 1 thou = 1000 rupees
hun_factor = 100  # 1 hun = 100 rupees

# Function to preprocess asset values
def preprocess_assets(value):
    if 'Crore' in str(value):
        return float(value.replace(' Crore+', '')) * crore_factor
    elif 'Lac' in str(value):
        return float(value.replace(' Lac+', '')) * lac_factor
    elif 'Thou' in str(value):
        return float(value.replace(' Thou+', '')) * thou_factor
    elif 'Hund' in str(value):
        return float(value.replace(' Hund+', '')) * hun_factor
    else:
        return float(value)

# Preprocess 'Total Assets' and 'Liabilities' columns
train_data['Total Assets'] = train_data['Total Assets'].apply(preprocess_assets)
train_data['Liabilities'] = train_data['Liabilities'].apply(preprocess_assets)

test_data['Total Assets'] = test_data['Total Assets'].apply(preprocess_assets)
test_data['Liabilities'] = test_data['Liabilities'].apply(preprocess_assets)

# Feature scaling
scaler = StandardScaler()
numerical_features = ['Criminal Case', 'Total Assets', 'Liabilities']
train_data[numerical_features] = scaler.fit_transform(train_data[numerical_features])
test_data[numerical_features] = scaler.transform(test_data[numerical_features])

# Define X_train, y_train, X_test
X_train = train_data.drop(columns=['Education'])
y_train = train_data['Education']

X_test = test_data.copy()  # Since there is no 'Education' column, we copy the test_data as is

# Separate numerical and categorical features
X_train_numeric = X_train.select_dtypes(include=[np.number])
X_test_numeric = X_test.select_dtypes(include=[np.number])


In [53]:
# Define X_train, y_train, X_test
X_train = train_data.drop(columns=['Education'])
y_train = train_data['Education']

X_test = test_data.copy()  # Since there is no 'Education' column, we copy the test_data as is

# Separate numerical and categorical features
X_train_numeric = X_train.select_dtypes(include=[np.number])
X_test_numeric = X_test.select_dtypes(include=[np.number])


In [54]:
# Define categorical columns
categorical_columns = ['Party', 'state']

# One-hot encode categorical features
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[categorical_columns]))
X_test_encoded = pd.DataFrame(encoder.transform(X_test[categorical_columns]))

# Reset index
X_train_encoded.reset_index(drop=True, inplace=True)
X_test_encoded.reset_index(drop=True, inplace=True)

# Combine encoded categorical features with numerical features
X_train_scaled = pd.concat([X_train_numeric, X_train_encoded], axis=1)
X_test_scaled = pd.concat([X_test_numeric, X_test_encoded], axis=1)

# Convert column names to strings
X_train_scaled.columns = X_train_scaled.columns.astype(str)
X_test_scaled.columns = X_test_scaled.columns.astype(str)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)


In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

model = RandomForestClassifier(n_estimators=398, random_state=47)

model.fit(X_train_scaled, y_train)



RandomForestClassifier(n_estimators=398, random_state=47)

In [56]:
# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Create a DataFrame with the ID column and the predicted values
predicted_data = pd.DataFrame({'ID': test_data['ID'], 'Predicted_Education': y_pred})

# Display the DataFrame
print(predicted_data)


        ID    Predicted_Education
0        0               Graduate
1        1  Graduate Professional
2        2               Graduate
3        3          Post Graduate
4        4               Graduate
...    ...                    ...
1369  1369          Post Graduate
1370  1370          Post Graduate
1371  1371          Post Graduate
1372  1372  Graduate Professional
1373  1373              12th Pass

[1374 rows x 2 columns]


In [57]:

# Create a new DataFrame with the ID column and the predicted values
submission_df = pd.DataFrame({'ID': test_data['ID'], 'Education': y_pred})

# Save the DataFrame as a CSV file
submission_df.to_csv('submission.csv', index=False)
